In [ ]:
import os
import json
import glob
import numpy as np
import pandas as pd
from astroquery.ipac.ned import Ned
from wiserep_api.properties import get_target_property

In [21]:
def load_z_dict():
    """Loads the dictionary with the redshift values.
    """
    redshift_file = 'redshifts.json'
    if os.path.isfile(redshift_file) is True:
        with open(redshift_file, 'r') as fp:
            z_dict = json.load(fp)
    else:
        z_dict = {'name':[], 'z':[], 'source':[], 'host_name':[]}
        
    return z_dict

def save_z_dict(z_dict):
    """Saves the dictionary with the redshift values. 
    """
    with open('redshifts.json', 'w') as outfile:
        json.dump(z_dict, outfile)
        
def get_redshift(target, overwrite=False):
    """Obtains the redshift for the given target.
    """
    z_dict = load_z_dict()
    if target in z_dict['name'] and overwrite is False:
        # quit
        return None
    
    host_name = get_target_property(target, 'host')
    if host_name!='':
        try:
            z_table = Ned.get_table(host_name, table='redshifts')
            z = z_table['Published Redshift'].value[0]
            source = 'host'
        except Exception as exc:
            print(f'{target} ({host_name}): {exc}')
            # quit
            return None
    else:
        z = np.nan
        source = ''

    if target not in z_dict['name']:
        z_dict['name'].append(target)
        z_dict['z'].append(z)
        z_dict['source'].append(source)
        z_dict['host_name'].append(host_name)

    elif target in z_dict['name'] and overwrite is True:
        target_id = z_dict['name'].index(target)

        z_dict['z'][target_id] = z
        z_dict['source'][target_id] = source
        z_dict['host_name'][target_id] = host_name
        
    save_z_dict(z_dict)

In [22]:
peak_spectra = glob.glob('peak_spectra/*')
targets = []
for file in peak_spectra:
    target = os.path.basename(file).split('_')[0]
    if target not in targets:
        targets.append(target)
        
len(targets)

1890

In [ ]:
for target in targets:
    get_redshift(target)

2004as (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2006bt (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2003ek (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2008bc (A093831-6358): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2008bf (NGC 4055): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
2007qe (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpret

In [ ]:
z_dict = load_z_dict()
z_df = pd.DataFrame(z_dict)
z_df.to_csv('redshifts.csv', index=False)